In [ ]:
import numpy as np
import pandas as pd
import time 
import random
from sklearn.model_selection import train_test_split
from sklearn import datasets, preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ds=pd.read_csv('/content/drive/MyDrive/diabetes.csv')
ds.head(15)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


# Import Data

In [ ]:

x = ds.drop(['Outcome'], axis = 1).values
y = ds['Outcome'].values
y = y.reshape(y.shape[0], 1)
#Spliting Data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=0)
#Standardize data
from sklearn.preprocessing import StandardScaler 
scale = StandardScaler()
x_train_std = scale.fit_transform(x_train)
x_test_std = scale.transform(x_test)


In [ ]:
print(y_train[0][0])

0


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

# Feed Forward

In [ ]:
def FF(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer

In [ ]:
NIL=8
HL=18
OL=1
#144+18+19=181

In [ ]:
def MLP(solution):
      fitness=0
      w=(NIL*HL)+(HL*OL)
      b=(NIL*HL)+HL+(HL*OL)+OL
      W=solution[0:w]
      B=solution[w:b]
      for idx,x in enumerate(x_train_std):
        actualValue=FF(HL,OL,W,B,x)

        fitness+=pow((actualValue[0]-y_train[idx][0]),2)

      fitness=fitness/len(x_train)
      
      return fitness

# HHO Algorithm

In [ ]:
import random
import numpy
import math
import time

class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0

def HHO(objf, lb, ub, dim, SearchAgents_no, Max_iter):

    # dim=30
    # SearchAgents_no=50
    # lb=-100
    # ub=100
    # Max_iter=500

    # initialize the location and Energy of the rabbit
    Rabbit_Location = numpy.zeros(dim)
    Rabbit_Energy = float("inf")  # change this to -inf for maximization problems

    if not isinstance(lb, list):
        lb = [lb for _ in range(dim)]
        ub = [ub for _ in range(dim)]
    lb = numpy.asarray(lb)
    ub = numpy.asarray(ub)

    # Initialize the locations of Harris' hawks
    X = numpy.asarray(
        [x * (ub - lb) + lb for x in numpy.random.uniform(0, 1, (SearchAgents_no, dim))]
    )

    # Initialize convergence
    convergence_curve = numpy.zeros(Max_iter)

    ############################
    s = solution()

    print('HHO is now tackling  "' + objf.__name__ + '"')

    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    ############################

    t = 0  # Loop counter

    # Main loop
    while t < Max_iter:
        for i in range(0, SearchAgents_no):

            # Check boundries

            X[i, :] = numpy.clip(X[i, :], lb, ub)

            # fitness of locations
            fitness = objf(X[i, :])

            # Update the location of Rabbit
            if fitness < Rabbit_Energy:  # Change this to > for maximization problem
                Rabbit_Energy = fitness
                Rabbit_Location = X[i, :].copy()

        E1 = 2 * (1 - (t / Max_iter))  # factor to show the decreaing energy of rabbit

        # Update the location of Harris' hawks
        for i in range(0, SearchAgents_no):

            E0 = 2 * random.random() - 1  # -1<E0<1
            Escaping_Energy = E1 * (
                E0
            )  # escaping energy of rabbit Eq. (3) in the paper

            # -------- Exploration phase Eq. (1) in paper -------------------

            if abs(Escaping_Energy) >= 1:
                # Harris' hawks perch randomly based on 2 strategy:
                q = random.random()
                rand_Hawk_index = math.floor(SearchAgents_no * random.random())
                X_rand = X[rand_Hawk_index, :]
                if q < 0.5:
                    # perch based on other family members
                    X[i, :] = X_rand - random.random() * abs(
                        X_rand - 2 * random.random() * X[i, :]
                    )

                elif q >= 0.5:
                    # perch on a random tall tree (random site inside group's home range)
                    X[i, :] = (Rabbit_Location - X.mean(0)) - random.random() * (
                        (ub - lb) * random.random() + lb
                    )

            # -------- Exploitation phase -------------------
            elif abs(Escaping_Energy) < 1:
                # Attacking the rabbit using 4 strategies regarding the behavior of the rabbit

                # phase 1: ----- surprise pounce (seven kills) ----------
                # surprise pounce (seven kills): multiple, short rapid dives by different hawks

                r = random.random()  # probablity of each event

                if (
                    r >= 0.5 and abs(Escaping_Energy) < 0.5
                ):  # Hard besiege Eq. (6) in paper
                    X[i, :] = (Rabbit_Location) - Escaping_Energy * abs(
                        Rabbit_Location - X[i, :]
                    )

                if (
                    r >= 0.5 and abs(Escaping_Energy) >= 0.5
                ):  # Soft besiege Eq. (4) in paper
                    Jump_strength = 2 * (
                        1 - random.random()
                    )  # random jump strength of the rabbit
                    X[i, :] = (Rabbit_Location - X[i, :]) - Escaping_Energy * abs(
                        Jump_strength * Rabbit_Location - X[i, :]
                    )

                # phase 2: --------performing team rapid dives (leapfrog movements)----------

                if (
                    r < 0.5 and abs(Escaping_Energy) >= 0.5
                ):  # Soft besiege Eq. (10) in paper
                    # rabbit try to escape by many zigzag deceptive motions
                    Jump_strength = 2 * (1 - random.random())
                    X1 = Rabbit_Location - Escaping_Energy * abs(
                        Jump_strength * Rabbit_Location - X[i, :]
                    )
                    X1 = numpy.clip(X1, lb, ub)

                    if objf(X1) < fitness:  # improved move?
                        X[i, :] = X1.copy()
                    else:  # hawks perform levy-based short rapid dives around the rabbit
                        X2 = (
                            Rabbit_Location
                            - Escaping_Energy
                            * abs(Jump_strength * Rabbit_Location - X[i, :])
                            + numpy.multiply(numpy.random.randn(dim), Levy(dim))
                        )
                        X2 = numpy.clip(X2, lb, ub)
                        if objf(X2) < fitness:
                            X[i, :] = X2.copy()
                if (
                    r < 0.5 and abs(Escaping_Energy) < 0.5
                ):  # Hard besiege Eq. (11) in paper
                    Jump_strength = 2 * (1 - random.random())
                    X1 = Rabbit_Location - Escaping_Energy * abs(
                        Jump_strength * Rabbit_Location - X.mean(0)
                    )
                    X1 = numpy.clip(X1, lb, ub)

                    if objf(X1) < fitness:  # improved move?
                        X[i, :] = X1.copy()
                    else:  # Perform levy-based short rapid dives around the rabbit
                        X2 = (
                            Rabbit_Location
                            - Escaping_Energy
                            * abs(Jump_strength * Rabbit_Location - X.mean(0))
                            + numpy.multiply(numpy.random.randn(dim), Levy(dim))
                        )
                        X2 = numpy.clip(X2, lb, ub)
                        if objf(X2) < fitness:
                            X[i, :] = X2.copy()

        convergence_curve[t] = Rabbit_Energy
        if t % 1 == 0:
            print(
                [
                    "At iteration "
                    + str(t)
                    + " the best fitness is "
                    + str(Rabbit_Energy)
                ]
            )
        t = t + 1

    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = convergence_curve
    s.optimizer = "HHO"
    s.objfname = objf.__name__
    s.best = Rabbit_Energy
    s.bestIndividual = Rabbit_Location

    return s


def Levy(dim):
    beta = 1.5
    sigma = (
        math.gamma(1 + beta)
        * math.sin(math.pi * beta / 2)
        / (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))
    ) ** (1 / beta)
    u = 0.01 * numpy.random.randn(dim) * sigma
    v = numpy.random.randn(dim)
    zz = numpy.power(numpy.absolute(v), (1 / beta))
    step = numpy.divide(u, zz)
    return step

In [ ]:
A=HHO(MLP, -1, 1, 181, 50, 1000)


HHO is now tackling  "MLP"
['At iteration 0 the best fitness is 0.2099652823649412']
['At iteration 1 the best fitness is 0.20141538772561493']
['At iteration 2 the best fitness is 0.20101971811998848']
['At iteration 3 the best fitness is 0.20101971811998848']
['At iteration 4 the best fitness is 0.201015130511172']
['At iteration 5 the best fitness is 0.201015130511172']
['At iteration 6 the best fitness is 0.20101169587422227']
['At iteration 7 the best fitness is 0.20101169587422227']
['At iteration 8 the best fitness is 0.20101169587422227']
['At iteration 9 the best fitness is 0.20098409572420908']
['At iteration 10 the best fitness is 0.20098409572420908']
['At iteration 11 the best fitness is 0.20098409572420908']
['At iteration 12 the best fitness is 0.20098409572420908']
['At iteration 13 the best fitness is 0.20098409572420908']
['At iteration 14 the best fitness is 0.20098409572420908']
['At iteration 15 the best fitness is 0.20098409572420908']
['At iteration 16 the best f

In [ ]:
print(A.convergence[999])

0.14419254304651696


In [ ]:
solution=A.bestIndividual
print(solution)

[-0.43617033 -0.89053164  0.99542283  0.94646308  0.55268408  0.76518628
 -0.24630927  0.93611489  0.42996044  0.81284283 -0.75264409  0.26907013
 -0.05447439  0.87259597  0.42813596  1.         -0.93254589 -0.31198533
  0.54519284 -0.32521294 -0.58502368 -0.47706973 -0.17661952 -0.21034978
 -0.65707688 -0.37881497  0.98585715 -0.95430543 -0.47276714  0.71349795
  0.24848931  0.98825575 -0.04120832 -0.35521794  0.01499072 -0.06257952
 -0.21977773  0.97727064 -0.73442946 -0.96114095 -0.71594348  0.71006672
 -0.18402231  0.53486383  0.88893737  1.          0.89176785 -0.5085941
  0.46587602 -0.2720621  -0.80147539  0.65488022  1.         -0.99521897
 -0.56635984  1.         -0.95774525 -0.91671875 -0.30416651 -0.77036806
 -0.79295136  0.99965299  0.81463492  0.67552919  0.47458496 -0.6895661
 -0.12108055  0.6426682   0.43024362 -0.33700916  0.05337691  0.08992876
  0.97486842 -0.62701659 -0.07331574  0.97944904 -0.41857549 -0.03105783
  0.9394183   0.61230631 -0.35705542  0.70023905  0.9

In [ ]:
def Test(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer




In [ ]:
min_key=min(B.keys(), key=(lambda k: B[k]))

In [ ]:
w=(NIL*HL)+(HL*OL)
b=(NIL*HL)+HL+(HL*OL)+OL    
W=solution[0:w]
B=solution[w:b]
print(len(W))
print(B)

162
[-0.87823264  0.49776684  0.93970759 -0.67462265 -0.07166569  0.84701433
 -0.84584934  0.45769606 -0.95835284  0.28428188 -0.80293612  0.41559904
 -0.43792979 -0.94485698  0.68785107  0.88895201 -0.85216654 -0.41950391
 -0.85408455]


In [ ]:
a=Test(HL,OL,W,B,x_test_std[7])
print(a)


[1.]


In [ ]:
R=0
for i in range(len(y_train)):
  p=Test(HL,OL,W,B,x_train_std[i])
  if y_train[i][0]==0:
    if p[0]<0.5:
      R+=1
  if y_train[i][0]==1:
    if p[0]>0.5:
      R+=1


In [ ]:
print((R*100)/len(y_train))

78.47222222222223


In [ ]:
def prediction(HL,OL,W,B,x):
  predict=[]
  s=[]
  for i in range(len(x)):
    a=Test(HL,OL,W,B,x[i])
    if a[0]>0.5:
      predict.append(1)
    else:
      predict.append(0)
    
    
  
  return predict

In [ ]:
pred=prediction(HL,OL,W,B,x_test_std)
print(f"Predict :{pred}")
y_true = sum(y_test.tolist(), [])
print(f"true y:{y_true}")

acc=0
for i in range(len(y_test)):
  if pred[i]==y_true[i]:
    acc+=1

print(f"accuracy:{acc/len(y_test)*100}%")

Predict :[1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
true y:[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1

In [ ]:
p=[[2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173583065964e-08], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859951413703e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999997594005218, 1.7827926890587977e-14], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3777059110728068e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3441849876730426e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [2.3443797239678989e-07, 1.0], [0.999997046667274, 1.0], [3.4230559529451456e-19, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.7353786067445575e-13, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998619, 7.412144614465116e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [3.2958383525020503e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537748567615e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.735378606784197e-13, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859963200897e-07, 1.0], [1.0, 7.49479046619701e-09], [2.730081969202577e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [2.824682922217676e-13, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.494780457447032e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.2482862984116327e-18, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.494790466200846e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.7601836872318313e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999999998, 3.6456980727045307e-10], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [3.561895408985392e-19, 1.0], [0.9999999999998617, 7.413141488936635e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.999997046667274, 1.0], [1.0, 7.49479046619701e-09], [2.3443797226534723e-07, 1.0], [0.9999999999997193, 1.377389407325726e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.735371990190059e-13, 1.0], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [1.0, 5.790537731467655e-12], [0.9999999954925718, 1.1219721194964509e-13], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.735378606754956e-13, 1.0], [2.344379723967903e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.7601861964494985e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 1.4646590413208634e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.416891915703537e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [4.7601859947178284e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997216, 1.382760290185998e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.494787237171255e-09], [0.9999997593963287, 1.7827783200580683e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 5.790537731467655e-12], [5.186223429698876e-06, 1.0], [1.0, 7.49479046619701e-09], [3.2963276408309863e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.760185995141387e-07, 1.0], [0.999999759396326, 1.7827783109247703e-14], [2.3443797239678989e-07, 1.0], [2.3443797239533066e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [3.539949630789169e-14, 1.0], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071096696e-17], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.399724176787439e-09], [3.2958381859094635e-19, 1.0], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.736071297671768e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.494790466196905e-09], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997597973544, 1.784154246233934e-14], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773916445774456e-17], [1.0, 5.7905377175175346e-12], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105640535e-14], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [4.760185994289154e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997594030715, 1.782801427057929e-14], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071148567e-17], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998492, 1.8358616739546913e-17], [2.3443797239678989e-07, 1.0], [2.3443792937421956e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.494790466200632e-09], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [0.9999999999997193, 1.3773894071084169e-17], [0.9999999999997193, 1.3773894071086518e-17], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.8247268646226163e-13, 1.0], [4.760185922590398e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.493736721731287e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995142792e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [3.295837933196582e-19, 1.0], [1.0, 5.7905377314674905e-12], [1.0, 5.790539620623675e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.492433652437726e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [5.0873878446689e-18, 1.0]]
for i in range(len(pred)):
  if p[i][0] and p[i][1] >0.5:
    print(i)
  